In [1]:
# import our modules
import re
import requests
import unicodedata
import numpy as np
from bs4 import BeautifulSoup as bs

In [2]:
# set our root url
root_url = "http://www.spotrac.com/nfl"

In [3]:
# request page using a "GET" request
r = requests.get(root_url)
# actually get the text of the page (in this case, HTML!)
page_text = r.text
# store the HTML in a BS object; we'll use the "lxml" parsing system
soup = bs(page_text, "lxml")
# we can actually see our web page stored in the BS object using one of the BS methods
# for example, here's a sample 400 characters...
print soup.prettify()[7600:8000]

ol-xs-4 text-right">
       <a href="#" id="nav-expander" onclick="return false;">
        <span class="fa fa-bars" style="z-index:1000000;position:relative;">
        </span>
       </a>
      </div>
      <!-- end:col -->
     </div>
     <!-- end:row -->
     <!-- start:sidr -->
     <div id="sidr">
      <form action="https://www.spotrac.com/redirect/search/" id="search-form-mobile" method="po


In [4]:
links = soup.find_all('a')
temp = []
for link in links:
    if (link.text[2:] == "Positional Spending"):
        temp.append(link.get('href'))
links=temp
links.remove("http://www.spotrac.com/nfl/los-angeles-chargers/positional/")
links.insert(26,"http://www.spotrac.com/nfl/san-diego-chargers/positional/")
print(links)
teams=[]
for l in links:
    team=l[::-1][12:]
    team=team[:team.find("/")][::-1]
    team=team.replace("-", " ")
    teams.append(team.title())
teams[27]="San Francisco 49ers"
newlinks=links[:]
newteams=teams[:]
oldrams="http://www.spotrac.com/nfl/st.-louis-rams/positional/"
oldlinks=links[:]
oldlinks.remove("http://www.spotrac.com/nfl/los-angeles-rams/positional/")
oldlinks.insert(28, oldrams)
oldramsname="St. Louis Rams"
oldteams=teams[:]
oldteams.remove("Los Angeles Rams")
oldteams.insert(28, oldramsname)
print(oldteams)

['http://www.spotrac.com/nfl/arizona-cardinals/positional/', 'http://www.spotrac.com/nfl/atlanta-falcons/positional/', 'http://www.spotrac.com/nfl/baltimore-ravens/positional/', 'http://www.spotrac.com/nfl/buffalo-bills/positional/', 'http://www.spotrac.com/nfl/carolina-panthers/positional/', 'http://www.spotrac.com/nfl/chicago-bears/positional/', 'http://www.spotrac.com/nfl/cincinnati-bengals/positional/', 'http://www.spotrac.com/nfl/cleveland-browns/positional/', 'http://www.spotrac.com/nfl/dallas-cowboys/positional/', 'http://www.spotrac.com/nfl/denver-broncos/positional/', 'http://www.spotrac.com/nfl/detroit-lions/positional/', 'http://www.spotrac.com/nfl/green-bay-packers/positional/', 'http://www.spotrac.com/nfl/houston-texans/positional/', 'http://www.spotrac.com/nfl/indianapolis-colts/positional/', 'http://www.spotrac.com/nfl/jacksonville-jaguars/positional/', 'http://www.spotrac.com/nfl/kansas-city-chiefs/positional/', 'http://www.spotrac.com/nfl/los-angeles-rams/positional/',

In [6]:
years = range(2013, 2017)
database = np.zeros(shape=(3584,7),dtype="a30")
counter = 0
for year in years:
    if (year < 2016):
        links = oldlinks
        teams = oldteams
    else:
        links = newlinks
        teams = newteams
    for i in range(len(links)):
        r = requests.get(links[i] + str(year) + "/full-cap/")
        soup = bs(r.text, "lxml")
        data = soup.find_all('td')
        for j in range(len(data)):
            if j % 5 != 0:
                continue
            else:
                database[counter]=[str(year), newteams[i], str(data[j].text), str(data[j+1].text), str(data[j+2].text), str(data[j+3].text), str(data[j+4].text)]
                counter += 1
np.savetxt("nfldata.txt",database,delimiter="\t",fmt="%s")
        